In [5]:
torch.cuda.is_available()

True

In [18]:
HOME_PATH = "/home/223C1851Rodrigo" # path where you cloned musicfm

import os
import sys
import torch

sys.path.append(HOME_PATH)
from musicfm.model.musicfm_25hz import MusicFM25Hz


# dummy audio (30 seconds, 24kHz)
wav = (torch.rand(4, 24000 * 30) - 0.5) * 2

# load MusicFM
musicfm = MusicFM25Hz(
    is_flash=False,
    stat_path=os.path.join(HOME_PATH, "musicfm", "data", "msd_stats.json"),
    model_path=os.path.join(HOME_PATH, "musicfm", "data", "pretrained_msd.pt"),
)

# to GPUs Esto sigue matándo el código
# wav = wav.cuda()
# musicfm = musicfm.cuda()

# get embeddings
musicfm.eval()
emb = musicfm.get_latent(wav, layer_ix=7)
##can expect better performance in downstream tasks by fine-tuning the foundation model. 
##In this scenario, employ musicfm.train() and extract the final embeddings by setting layer_ix=12

In [19]:
print( emb)

tensor([[[ 1.4970e+00,  8.7504e-01, -8.3683e-01,  ...,  5.7270e-01,
           4.1638e-01,  8.2667e-01],
         [ 1.3080e+00,  1.3058e+00, -6.6828e-01,  ...,  1.2510e-01,
          -1.4943e-01, -1.4913e+00],
         [ 1.3017e+00,  8.8395e-01, -5.9292e-01,  ...,  5.9626e-01,
           1.2431e-03, -3.1398e-01],
         ...,
         [ 1.2605e+00,  1.0016e+00, -1.5569e-02,  ..., -2.4732e-01,
          -1.4812e-01, -1.9853e+00],
         [ 1.4641e+00,  6.5648e-01, -3.0902e-01,  ..., -4.7485e-02,
          -1.3053e-02, -1.8475e+00],
         [ 1.2501e+00,  4.6683e-01, -3.4340e-01,  ..., -7.8366e-02,
           7.0811e-01, -7.8484e-01]],

        [[ 1.7316e+00,  9.4696e-01, -9.2006e-01,  ...,  8.2110e-02,
           8.8368e-01,  1.6495e+00],
         [ 1.1566e+00,  1.4032e+00, -5.0205e-01,  ...,  2.3462e-01,
           1.8088e-02, -1.7308e+00],
         [ 1.3888e+00,  1.4740e+00, -4.8198e-01,  ...,  6.0656e-01,
          -8.7624e-02, -1.0669e+00],
         ...,
         [ 1.6265e+00,  5

In [22]:
print(musicfm.get_predictions(wav))

({'melspec_2048': tensor([[[ -9.5317,  -9.1490,  -1.3315,  ...,  -3.0235,  -1.1616,  -3.3108],
         [ -8.5902,  -9.1430,  -0.6352,  ...,  -3.9792,   0.7234,  -2.7171],
         [ -8.9087,  -8.9148,  -0.4804,  ...,  -4.1912,   0.4623,  -3.0079],
         ...,
         [ -9.5038,  -9.2422,   3.7032,  ...,  -6.0093,  -1.0466,  -4.7160],
         [ -9.6420,  -9.1740,   3.0789,  ...,  -6.2738,  -1.4350,  -5.2572],
         [-10.5684,  -8.7596,  -0.7713,  ...,  -5.4823,  -4.1541,  -5.8397]],

        [[ -9.1309,  -9.1345,  -0.8058,  ...,  -3.2285,  -1.3119,  -3.5730],
         [ -7.8263,  -9.1324,   0.6957,  ...,  -4.0041,   0.5194,  -3.2448],
         [ -8.3673,  -9.4472,   1.1538,  ...,  -4.4232,   0.2801,  -3.2979],
         ...,
         [ -8.3538,  -7.4180,   6.4661,  ...,  -4.4380,  -1.5450,  -3.5117],
         [ -7.8630,  -7.0570,   5.2318,  ...,  -4.5929,  -1.1048,  -3.1822],
         [ -8.2123,  -6.7555,   0.8293,  ...,  -3.9182,  -2.4828,  -3.0776]],

        [[ -9.1850,  -9.65

In [28]:
targets=musicfm.get_targets(wav)
print(targest)

AttributeError: 'MusicFM25Hz' object has no attribute 'quantizer_melspec_2048'

In [24]:
print(wav)

tensor([[-0.6459,  0.7770,  0.7820,  ..., -0.5973, -0.9543,  0.6255],
        [-0.6000, -0.1187,  0.1652,  ...,  0.0680,  0.3178, -0.8991],
        [-0.5846,  0.7040,  0.0812,  ...,  0.5050,  0.9705, -0.3003],
        [ 0.9353, -0.8581,  0.4357,  ...,  0.4616,  0.8442, -0.1013]])


In [26]:
print(musicfm.forward(wav))

AttributeError: 'MusicFM25Hz' object has no attribute 'quantizer_melspec_2048'

In [6]:
from torch import nn

# Sequence-level representation
seq_emb = emb.mean(-1) # (batch, time, channel) -> (batch, channel)

# Frame-level representation
"""
	n_frame = desired_temporal_resolution * sequence_length_in_sec
	300 frames = 10Hz * 30s in this example
	As a result, the sequence length becomes from 750 (25Hz * 30s) to 300
"""
n_frame = 300 
token_emb = nn.AdaptiveAvgPool1d(n_frame)(emb) # (batch, time, channel) -> (batch, time', channel)

In [7]:
print(token_emb)

tensor([[[ 0.3510, -0.4139, -0.1807,  ..., -0.2369, -0.1801,  0.2973],
         [ 0.4739,  0.6454,  0.3148,  ...,  0.1454,  0.0049, -0.3172],
         [ 0.3800,  0.2692,  0.2592,  ...,  0.0970, -0.1315, -0.4930],
         ...,
         [ 0.3037,  0.1951, -0.1616,  ..., -0.1453, -0.0506, -0.9505],
         [ 0.1489,  0.1421, -0.3329,  ..., -0.2695, -0.2109, -0.2841],
         [ 0.1653,  0.0107, -0.2363,  ..., -0.0394, -0.8655, -0.6815]],

        [[ 0.3979, -0.4953, -0.1565,  ..., -0.1574,  0.1921,  0.5931],
         [ 0.4800,  0.5787,  0.3373,  ...,  0.0119, -0.1483, -0.6937],
         [ 0.3301,  0.5515,  0.3870,  ...,  0.0931,  0.0120, -0.5440],
         ...,
         [ 0.1943,  0.4712, -0.2211,  ..., -0.4312, -0.5710, -0.9467],
         [ 0.2163,  0.2753, -0.2028,  ..., -0.2339,  0.0894, -0.7058],
         [ 0.2666, -0.1362, -0.2324,  ..., -0.0580, -0.5955, -0.8119]],

        [[ 0.3757, -0.6988, -0.2238,  ..., -0.2688, -0.1398,  0.4065],
         [ 0.5015,  0.7275,  0.3392,  ..., -0

In [25]:
print(len(token_emb[1][1]))

300


In [26]:
print(token_emb[1][1])

tensor([ 0.4839,  0.6763,  0.3017, -0.1981,  0.3454, -0.0965, -0.2128,  0.0924,
        -0.2704, -0.0628, -0.2055,  0.0517,  0.4736,  0.1387,  0.6966,  0.1419,
         0.4394,  0.4698,  0.5114, -0.0089,  0.7142, -0.1297, -0.0428, -0.2391,
         0.6026,  0.0044,  0.2988,  0.0021,  0.7418,  0.4120,  0.1208, -0.0217,
         0.3142,  0.1719,  0.4459,  0.5402,  0.6761, -0.2327,  0.0397,  0.3398,
         0.0221,  0.0334, -0.1831,  0.1684, -0.4952, -0.3983, -0.4923,  0.1264,
        -0.4679,  0.0443,  0.3705,  0.5610, -0.0680, -0.1897,  0.3850,  0.3313,
        -0.0060, -0.0885,  0.0771, -0.1030, -0.1579,  0.2765,  0.6296,  0.0412,
        -0.2127,  0.1762, -0.9725, -0.2633,  0.1889, -0.3095,  0.0812,  0.5513,
        -0.2967, -0.3030,  0.1645, -0.3362, -0.3619, -0.1928, -0.7751,  0.2925,
         0.1932,  0.0928, -0.1107, -0.1596,  0.0873, -0.5905,  0.9318,  1.1410,
         0.0560, -0.1223,  0.3386, -0.6838, -0.2405, -0.0152,  0.8132, -0.8153,
        -0.0194, -0.7206, -0.2756, -0.54

In [27]:
pip install mir_eval

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 KB 21.8 MB/s eta 0:00:00
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100722 sha256=afab59b71ddb3789988fd902f9c1df588f158484dd38465b95ed9a5e192a371e
  Stored in directory: /home/223C1851Rodrigo/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built mir_eval
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [9]:
print(musicfm.forward)

<bound method MusicFM25Hz.forward of MusicFM25Hz(
  (preprocessor_melspec_2048): MelSTFT(
    (mel_stft): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
    (amplitude_to_db): AmplitudeToDB()
  )
  (quantizer_melspec_2048_0): RandomProjectionQuantizer()
  (conv): Conv2dSubsampling(
    (conv): Sequential(
      (0): Res2dModule(
        (conv1): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv3): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Res2dModule(
        (conv1): Conv2d(512, 512, k